<a href="https://colab.research.google.com/github/Benjamon22/Restaurant-Recommendation-Engine/blob/main/Reviews_sentiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
file_path = '/content/drive/MyDrive/review_no_stopwords.csv'
import pandas as pd

In [3]:
chunks = []
chunksize = 100000

for chunk in pd.read_csv(file_path, chunksize=chunksize):
    chunks.append(chunk)

# Combine all chunks into one DataFrame
review_df = pd.concat(chunks, ignore_index=True)
review_df.head()

<ipython-input-3-44d0bf205f26>:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunksize):


,review_id,user_id,business_id,stars,useful,funny,cool,text,date,review_score,stopwords_removed
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0.0,0.0,0.0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11,0.0,decide eat aware going hours beginning end tri...
1,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1.0,0.0,1.0,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03,0.8,Wow Yummy different delicious favorite lamb cu...
2,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,1.0,0.0,1.0,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15,0.8,Cute interior owner gave tour upcoming patio r...
3,JrIxlS1TzJ-iCu79ul40cQ,eUta8W_HdHMXPzLBBZhL1A,04UD14gamNjLY0IDYVhHJg,1.0,1.0,2.0,1.0,I am a long term frequent customer of this est...,2015-09-23 23:10:31,1.2,long term frequent customer establishment went...
4,_ZeMknuYdlQcUqng_Im3yg,yfFzsLmaWF2d4Sr0UNbBgg,LHSTtnW3YHCeUkRDGyJOyw,5.0,2.0,0.0,0.0,Amazingly amazing wings and homemade bleu chee...,2015-08-07 02:29:16,1.2,Amazingly amazing wings homemade bleu cheese r...


In [4]:
review_df.shape

(4988736, 11)

In [5]:
food_keywords = {"food", "dish", "meal", "taste", "delicious", "menu", "flavor",
                 "cuisine", "plate", "appetizer", "dessert", "drink", "bowl",
                 "burger", "pizza", "soup", "steak", "cheese"
}
service_keywords = {"waiter", "waitress", "staff", "service", "friendly", "rude",
                    "attentive", "host", "manager", "customer", "attitude", "ass", "owner", "option"
}
ambience_keywords = {"ambience", "atmosphere", "environment", "lighting", "music",
                     "decor", "noise", "seating", "vibe", "clean", "interior", "crowded"
}
value_keywords = {
    "value", "price", "worth", "overpriced", "expensive", "cheap", "affordable",
    "cost", "deal", "portion", "serving", "size", "small", "huge", "tiny",
    "filling", "generous", "stingy", "money", "bang", "spent", "enough", "barely"
}

In [6]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
SIA = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [7]:
from tqdm import tqdm
tqdm.pandas()
def get_aspect_sentiment(text, keywords):
    if not isinstance(text, str):
        return None
    sentences = text.split('.')
    relevant = [s for s in sentences if any(word in s.lower() for word in keywords)]
    if not relevant:
        return None
    scores = [SIA.polarity_scores(s)['compound'] for s in relevant]
    return sum(scores) / len(scores) if scores else None

review_df['food_sentiment'] = review_df['stopwords_removed'].progress_apply(lambda x: get_aspect_sentiment(x, food_keywords))
review_df['service_sentiment'] = review_df['stopwords_removed'].progress_apply(lambda x: get_aspect_sentiment(x, service_keywords))
review_df['ambience_sentiment'] = review_df['stopwords_removed'].progress_apply(lambda x: get_aspect_sentiment(x, ambience_keywords))
review_df['value_sentiment'] = review_df['stopwords_removed'].progress_apply(lambda x: get_aspect_sentiment(x, value_keywords))
review_df.head()

100%|██████████| 4988736/4988736 [28:09<00:00, 2953.30it/s]


,review_id,user_id,business_id,stars,useful,funny,cool,text,date,review_score,stopwords_removed,food_sentiment,service_sentiment,ambience_sentiment,value_sentiment
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0.0,0.0,0.0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11,0.0,decide eat aware going hours beginning end tri...,0.6705,0.6705,NaN,0.6705
1,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1.0,0.0,1.0,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03,0.8,Wow Yummy different delicious favorite lamb cu...,0.9509,NaN,NaN,NaN
2,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,1.0,0.0,1.0,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15,0.8,Cute interior owner gave tour upcoming patio r...,0.9766,0.9766,0.9766,0.9766
3,JrIxlS1TzJ-iCu79ul40cQ,eUta8W_HdHMXPzLBBZhL1A,04UD14gamNjLY0IDYVhHJg,1.0,1.0,2.0,1.0,I am a long term frequent customer of this est...,2015-09-23 23:10:31,1.2,long term frequent customer establishment went...,NaN,0.5859,NaN,NaN
4,_ZeMknuYdlQcUqng_Im3yg,yfFzsLmaWF2d4Sr0UNbBgg,LHSTtnW3YHCeUkRDGyJOyw,5.0,2.0,0.0,0.0,Amazingly amazing wings and homemade bleu chee...,2015-08-07 02:29:16,1.2,Amazingly amazing wings homemade bleu cheese r...,0.9706,NaN,NaN,NaN


In [8]:
review_df[review_df['service_sentiment'] == review_df['service_sentiment'].min()]['stopwords_removed'].values[0]

'Beware bought roor glass pieces found later roor fake sell knock glass tell hand called confront admitted fake refused money knows fake pieces sell Stay away stores knock offs fakes knock offs fakes knock offs fakes knock offs fakes knock offs fakes knock offs fakes knock offs fakes knock offs fakes knock offs fakes knock offs fakes knock offs fakes knock offs fakesknock offs fakes knock offs fakesknock offs fakesknock offs fakesknock offs fakesknock offs fakesknock offs fakesknock offs fakesknock offs fakesknock offs fakesknock offs fakesknock offs fakesknock offs fakes knock offs fakes knock offs fakes knock offs fakes knock offs fakes knock offs fakes knock offs fakes knock offs fakes knock offs fakes knock offs fakes knock offs fakes knock offs fakes knock offs fakes knock offs fakes knock offs fakes knock offs fakes knock offs fakes knock offs fakes knock offs fakes knock offs fakes knock offs fakes knock offs fakes knock offs fakes knock offs fakes knock offs fakes knock offs fa

In [9]:
review_df[review_df['ambience_sentiment'] == review_df['ambience_sentiment'].min()]['stopwords_removed'].values[0]

'Pathetic limited buffet small portions bland food dry tasteless lousy service rude depressing dreary dull atmosphere ordered barbeque burger menu stated SMOTHERED barbeque sauce Swiss cheese DROP barbeque sauce Swiss cheese burger cold requested center burger raw rare actually raw hungry took bites disgusting tasteless bone dry lousy burger left rest burger extremely dry bland cold tasted old husband ordered Philly Cheese Steak skimpy looked awful husband said tasted like cold shit soft drink beverages watered colas look like iced tea diluted water place stingy cheap pathetic Needless disappointed honestly worst dining experience paid bill leave tip food service AWFUL burgers McDonalds taste million times better lot cheaper ask server times sour cream baked potato saw server small plastic cup sour cream person CONSUMED DIRTY plate sour cream stranger dirty plate totally grossed customer image disgusting nasty things scene Probably gave lousy leftovers customers shudder think LEFTOVER 

In [10]:
review_df[review_df['food_sentiment'] == review_df['food_sentiment'].min()]['stopwords_removed'].values[0]

'Pathetic limited buffet small portions bland food dry tasteless lousy service rude depressing dreary dull atmosphere ordered barbeque burger menu stated SMOTHERED barbeque sauce Swiss cheese DROP barbeque sauce Swiss cheese burger cold requested center burger raw rare actually raw hungry took bites disgusting tasteless bone dry lousy burger left rest burger extremely dry bland cold tasted old husband ordered Philly Cheese Steak skimpy looked awful husband said tasted like cold shit soft drink beverages watered colas look like iced tea diluted water place stingy cheap pathetic Needless disappointed honestly worst dining experience paid bill leave tip food service AWFUL burgers McDonalds taste million times better lot cheaper ask server times sour cream baked potato saw server small plastic cup sour cream person CONSUMED DIRTY plate sour cream stranger dirty plate totally grossed customer image disgusting nasty things scene Probably gave lousy leftovers customers shudder think LEFTOVER 

In [11]:
print(f"Empty values in food_sentiment: {int(review_df['food_sentiment'].isna().sum())}")
print(f"Empty values in service_sentiment: {int(review_df['service_sentiment'].isna().sum())}")
print(f"Empty values in ambience_sentiment: {int(review_df['ambience_sentiment'].isna().sum())}")
print(f"Empty values in value_sentiment: {int(review_df['value_sentiment'].isna().sum())}")

Empty values in food_sentiment: 994939
Empty values in service_sentiment: 2019531
Empty values in ambience_sentiment: 3797032
Empty values in value_sentiment: 3000781


In [17]:
import numpy as np

review_original = review_df.copy()
review_original.head(8)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,review_score,stopwords_removed,food_sentiment,service_sentiment,ambience_sentiment,value_sentiment
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0.0,0.0,0.0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11,0.0,decide eat aware going hours beginning end tri...,0.6705,0.6705,NaN,0.6705
1,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1.0,0.0,1.0,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03,0.8,Wow Yummy different delicious favorite lamb cu...,0.9509,NaN,NaN,NaN
2,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,1.0,0.0,1.0,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15,0.8,Cute interior owner gave tour upcoming patio r...,0.9766,0.9766,0.9766,0.9766
3,JrIxlS1TzJ-iCu79ul40cQ,eUta8W_HdHMXPzLBBZhL1A,04UD14gamNjLY0IDYVhHJg,1.0,1.0,2.0,1.0,I am a long term frequent customer of this est...,2015-09-23 23:10:31,1.2,long term frequent customer establishment went...,NaN,0.5859,NaN,NaN
4,_ZeMknuYdlQcUqng_Im3yg,yfFzsLmaWF2d4Sr0UNbBgg,LHSTtnW3YHCeUkRDGyJOyw,5.0,2.0,0.0,0.0,Amazingly amazing wings and homemade bleu chee...,2015-08-07 02:29:16,1.2,Amazingly amazing wings homemade bleu cheese r...,0.9706,NaN,NaN,NaN
5,pUycOfUwM8vqX7KjRRhUEA,59MxRhNVhU9MYndMkz0wtw,gebiRewfieSdtt17PTW6Zg,3.0,0.0,0.0,0.0,Had a party of 6 here for hibachi. Our waitres...,2016-07-25 07:31:06,0.0,party hibachi waitress brought separate sushi ...,0.9618,0.9618,NaN,0.9618
6,l3Wk_mvAog6XANIuGQ9C7Q,ZbqSHbgCjzVAqaa7NKWn5A,EQ-TZ2eeD_E0BHuvoaeG5Q,4.0,0.0,0.0,0.0,"Locals recommended Milktooth, and it's an amaz...",2015-08-19 14:31:45,0.0,Locals recommended Milktooth amazing jewel Ind...,NaN,NaN,NaN,NaN
7,XW_LfMv0fV21l9c6xQd_lw,9OAtfnWag-ajVxRbUTGIyg,lj-E32x9_FA7GmUrBGBEWg,4.0,0.0,0.0,0.0,Love going here for happy hour or dinner! Gre...,2014-06-27 22:44:01,0.0,Love going happy hour dinner Great patio fans ...,0.9382,NaN,NaN,NaN


In [ ]:
# Most reviews mentioned about food, but not so much mentioned about ambience or value of food.
# That, however, shows a sign of silence, so we are going to interpolate the missing sentiments based on the stars and review dates
review_sorted = review_df.sort_values(
    by=['stars', 'review_score', 'date'],
    ascending=[False, False, True]
)

# Interpolate sentiment columns
sentiment_cols = ['food_sentiment', 'service_sentiment', 'ambience_sentiment', 'value_sentiment']
review_sorted[sentiment_cols] = review_sorted[sentiment_cols].interpolate(method='linear', limit_direction='both')
# Sort the index back to avoid misalignment
review_sorted = review_sorted.sort_index()

In [19]:
print(f"Empty values in food_sentiment: {int(review_sorted['food_sentiment'].isna().sum())}")
print(f"Empty values in service_sentiment: {int(review_sorted['service_sentiment'].isna().sum())}")
print(f"Empty values in ambience_sentiment: {int(review_sorted['ambience_sentiment'].isna().sum())}")
print(f"Empty values in value_sentiment: {int(review_sorted['value_sentiment'].isna().sum())}")

Empty values in food_sentiment: 0
Empty values in service_sentiment: 0
Empty values in ambience_sentiment: 0
Empty values in value_sentiment: 0


In [20]:
review_sorted.head(8)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,review_score,stopwords_removed,food_sentiment,service_sentiment,ambience_sentiment,value_sentiment
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0.0,0.0,0.0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11,0.0,decide eat aware going hours beginning end tri...,0.67050,0.670500,0.948936,0.670500
1,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1.0,0.0,1.0,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03,0.8,Wow Yummy different delicious favorite lamb cu...,0.95090,0.960567,0.795800,0.795800
2,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,1.0,0.0,1.0,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15,0.8,Cute interior owner gave tour upcoming patio r...,0.97660,0.976600,0.976600,0.976600
3,JrIxlS1TzJ-iCu79ul40cQ,eUta8W_HdHMXPzLBBZhL1A,04UD14gamNjLY0IDYVhHJg,1.0,1.0,2.0,1.0,I am a long term frequent customer of this est...,2015-09-23 23:10:31,1.2,long term frequent customer establishment went...,-0.50310,0.585900,0.415359,-0.884950
4,_ZeMknuYdlQcUqng_Im3yg,yfFzsLmaWF2d4Sr0UNbBgg,LHSTtnW3YHCeUkRDGyJOyw,5.0,2.0,0.0,0.0,Amazingly amazing wings and homemade bleu chee...,2015-08-07 02:29:16,1.2,Amazingly amazing wings homemade bleu cheese r...,0.97060,0.946533,0.989309,0.972929
5,pUycOfUwM8vqX7KjRRhUEA,59MxRhNVhU9MYndMkz0wtw,gebiRewfieSdtt17PTW6Zg,3.0,0.0,0.0,0.0,Had a party of 6 here for hibachi. Our waitres...,2016-07-25 07:31:06,0.0,party hibachi waitress brought separate sushi ...,0.96180,0.961800,0.946467,0.961800
6,l3Wk_mvAog6XANIuGQ9C7Q,ZbqSHbgCjzVAqaa7NKWn5A,EQ-TZ2eeD_E0BHuvoaeG5Q,4.0,0.0,0.0,0.0,"Locals recommended Milktooth, and it's an amaz...",2015-08-19 14:31:45,0.0,Locals recommended Milktooth amazing jewel Ind...,0.61345,0.613450,0.949273,0.292025
7,XW_LfMv0fV21l9c6xQd_lw,9OAtfnWag-ajVxRbUTGIyg,lj-E32x9_FA7GmUrBGBEWg,4.0,0.0,0.0,0.0,Love going here for happy hour or dinner! Gre...,2014-06-27 22:44:01,0.0,Love going happy hour dinner Great patio fans ...,0.93820,0.904400,0.965022,0.917840


In [27]:
# Some values in the star column has string entries, convert them into floats
review_sorted['stars'] = pd.to_numeric(review_sorted['stars'], errors='coerce')
review_sorted[review_sorted['stars'].isna()]

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,review_score,stopwords_removed,food_sentiment,service_sentiment,ambience_sentiment,value_sentiment
2092131,We were welcomed at the door. The staff here w...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.5574,-0.7506,-0.5984,-0.7506
2092132,There were so many great menu choices and a lo...,2016-03-21 19:31:55,0.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.9716,0.9995,0.9349,0.9753


In [28]:
# Drop the 2 rows with NaN values
review_sorted = review_sorted.dropna(subset=['stars']).reset_index(drop=True)
review_sorted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4988734 entries, 0 to 4988733
Data columns (total 15 columns):
 #   Column              Dtype  
---  ------              -----  
 0   review_id           object 
 1   user_id             object 
 2   business_id         object 
 3   stars               float64
 4   useful              float64
 5   funny               float64
 6   cool                float64
 7   text                object 
 8   date                object 
 9   review_score        float64
 10  stopwords_removed   object 
 11  food_sentiment      float64
 12  service_sentiment   float64
 13  ambience_sentiment  float64
 14  value_sentiment     float64
dtypes: float64(9), object(6)
memory usage: 570.9+ MB


In [29]:
# We have now completed the sentiment analysis for the review dataset
review_sorted.to_csv('reviews_with_sentiments.csv', index=False)

In [30]:
!mv reviews_with_sentiments.csv /content/drive/MyDrive/